# BDH Model Training (Local Files)

This notebook trains the BDH model using the local `Books/` folder and `train.csv` in this repository.

**Steps:**
1. Install dependencies (optional, if not already installed)
2. Configure local paths
3. Load books + CSVs from local disk
4. Train the model
5. Save the trained model locally


In [7]:
# Install dependencies (optional)
from pathlib import Path
import sys
import subprocess

req = Path("requirements.txt")
if req.exists():
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-r", str(req)])
    print("✓ Installed dependencies from requirements.txt")
else:
    print("⚠️  requirements.txt not found next to the notebook")


✓ Installed dependencies from requirements.txt


## Step 1: Configure Local Paths


In [8]:
from pathlib import Path

# Assumes you run this notebook from: .../Long-Context-Consistency-of-LLM-KDSH-26/Main_System
NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = NOTEBOOK_DIR.parent

BOOKS_DIR = PROJECT_ROOT / "Books"
TRAIN_CSV_PATH = PROJECT_ROOT / "train.csv"
TEST_CSV_PATH = PROJECT_ROOT / "test.csv"  # optional

print(f"📁 Notebook dir : {NOTEBOOK_DIR}")
print(f"📁 Project root : {PROJECT_ROOT}")
print(f"📚 Books dir    : {BOOKS_DIR} {'✓' if BOOKS_DIR.exists() else '✗'}")
print(f"📋 train.csv    : {TRAIN_CSV_PATH} {'✓' if TRAIN_CSV_PATH.exists() else '✗'}")
print(f"📋 test.csv     : {TEST_CSV_PATH} {'✓' if TEST_CSV_PATH.exists() else '(optional)'}")

if not BOOKS_DIR.exists():
    raise FileNotFoundError(
        f"Books directory not found at {BOOKS_DIR}. "
        "Expected it at ../Books relative to Main_System."
    )

if not TRAIN_CSV_PATH.exists():
    raise FileNotFoundError(
        f"train.csv not found at {TRAIN_CSV_PATH}. "
        "Expected it at ../train.csv relative to Main_System."
    )


📁 Notebook dir : /home/aditya/kdsh2026/Long-Context-Consistency-of-LLM-KDSH-26/Main_System
📁 Project root : /home/aditya/kdsh2026/Long-Context-Consistency-of-LLM-KDSH-26
📚 Books dir    : /home/aditya/kdsh2026/Long-Context-Consistency-of-LLM-KDSH-26/Books ✓
📋 train.csv    : /home/aditya/kdsh2026/Long-Context-Consistency-of-LLM-KDSH-26/train.csv ✓
📋 test.csv     : /home/aditya/kdsh2026/Long-Context-Consistency-of-LLM-KDSH-26/test.csv ✓


## Step 2: Upload Core Files

Upload these Python files from your local machine:
- `core/bdh.py`
- `utils/bdh_narrative_builder.py`


## Step 3: Load Books and CSV from Google Drive


In [13]:
import sys
from pathlib import Path

import torch
import numpy as np
from tqdm import tqdm

# Make imports work whether CWD is Main_System or elsewhere
def _add_repo_paths_to_sys_path() -> None:
    start = Path.cwd().resolve()
    candidates = [start, *start.parents]
    project_root = None
    main_system_dir = None

    for base in candidates:
        # Case 1: running from repo root
        if (base / "Main_System" / "core" / "bdh.py").exists():
            project_root = base
            main_system_dir = base / "Main_System"
            break
        # Case 2: running from Main_System
        if (base / "core" / "bdh.py").exists() and (base / "utils" / "bdh_narrative_builder.py").exists():
            main_system_dir = base
            project_root = base.parent
            break

    if main_system_dir is None:
        raise FileNotFoundError(
            "Could not locate Main_System/core/bdh.py. "
            "Run the notebook from .../Main_System or open it from the repo."
        )

    core_dir = str((main_system_dir / "core").resolve())
    utils_dir = str((main_system_dir / "utils").resolve())
    if core_dir not in sys.path:
        sys.path.insert(0, core_dir)
    if utils_dir not in sys.path:
        sys.path.insert(0, utils_dir)


_add_repo_paths_to_sys_path()

from bdh import BDH, BDHConfig
from bdh_narrative_builder import _text_to_tokens

print(f"✓ PyTorch version: {torch.__version__}")
print(f"✓ CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✓ GPU: {torch.cuda.get_device_name(0)}")
    print(f"✓ GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")


✓ PyTorch version: 2.9.1+cu128
✓ CUDA available: True
✓ GPU: NVIDIA GeForce RTX 3050 Ti Laptop GPU
✓ GPU Memory: 4.1 GB


## Step 5: Train Model

This uses **MEMORY-EFFICIENT** training code to prevent crashes:
- ✅ Mixed precision training - **1.5-2x faster + less memory**
- ✅ `torch.compile()` - **2-3x faster** (optional, can disable if crashes)
- ✅ Random batches (no pre-allocation)
- ✅ CUDA optimizations
- ✅ Memory cleanup every 500 steps

**Training time: 20-35 minutes** on Colab GPU

**Settings (memory-safe):**
- `max_iters=3000` iterations
- `batch_size=16` (reduced to prevent OOM)
- `block_size=256` (reduced to prevent OOM)
- Token limit: 1M (prevents crashes)

**If session crashes:**
1. Reduce `batch_size=8` in the code below
2. Reduce `block_size=128`
3. Set `USE_COMPILE = False` in the function
4. Reduce `max_iters=2000` for faster completion


In [17]:
# Adapted to use local Books/ and a smaller model that fits on modest GPUs
import os
import sys
from contextlib import nullcontext
from pathlib import Path
import tempfile

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

# Ensure local imports work even if this cell is run first
def _add_repo_paths_to_sys_path() -> None:
    start = Path.cwd().resolve()
    candidates = [start, *start.parents]
    main_system_dir = None
    for base in candidates:
        if (base / "core" / "bdh.py").exists() and (base / "utils" / "bdh_narrative_builder.py").exists():
            main_system_dir = base
            break
        if (base / "Main_System" / "core" / "bdh.py").exists():
            main_system_dir = base / "Main_System"
            break
    if main_system_dir is None:
        raise FileNotFoundError("Could not locate Main_System/core/bdh.py for local imports")
    for p in [main_system_dir / "core", main_system_dir / "utils"]:
        ps = str(p.resolve())
        if ps not in sys.path:
            sys.path.insert(0, ps)


_add_repo_paths_to_sys_path()
import bdh

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

dtype = (
    "bfloat16"
    if torch.cuda.is_available() and torch.cuda.is_bf16_supported()
    else "float16"
 )  # 'float32', 'bfloat16', or 'float16'
ptdtype = {"float32": torch.float32, "bfloat16": torch.bfloat16, "float16": torch.float16}[dtype]

ctx = (
    torch.amp.autocast(device_type=device.type, dtype=ptdtype)
    if "cuda" in device.type
    else nullcontext()
)

scaler = torch.amp.GradScaler(device=device.type, enabled=(dtype == "float16"))

torch.manual_seed(1337)
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

gpu_mem_gb = None
if torch.cuda.is_available():
    gpu_mem_gb = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"Using device: {device} ({gpu_mem_gb:.1f} GB) with dtype {dtype}")
else:
    print(f"Using device: {device} with dtype {dtype}")

# --------------------
# Memory-safe defaults
# --------------------
is_small_gpu = gpu_mem_gb is not None and gpu_mem_gb <= 5.5

# The biggest memory driver in this model is the sparse width:
# N = (n_embd * mlp_internal_dim_multiplier) / n_head
# So to fit smaller GPUs, reduce BOTH n_embd and mlp_internal_dim_multiplier.
if is_small_gpu:
    default_block_size = 128
    default_batch_size = 2
    default_use_compile = 0
    default_n_embd = 128
    default_n_head = 2
    default_n_layer = 4
    default_mlp_mult = 16
else:
    default_block_size = 512
    default_batch_size = 16
    default_use_compile = 1
    default_n_embd = 256
    default_n_head = 4
    default_n_layer = 6
    default_mlp_mult = 128

BLOCK_SIZE = int(os.getenv("BDH_BLOCK_SIZE", str(default_block_size)))
BATCH_SIZE = int(os.getenv("BDH_BATCH_SIZE", str(default_batch_size)))
MAX_ITERS = int(os.getenv("BDH_MAX_ITERS", "3000"))
LEARNING_RATE = float(os.getenv("BDH_LR", "1e-3"))
WEIGHT_DECAY = float(os.getenv("BDH_WEIGHT_DECAY", "0.1"))
LOG_FREQ = int(os.getenv("BDH_LOG_FREQ", "100"))
USE_COMPILE = int(os.getenv("BDH_USE_COMPILE", str(default_use_compile))) == 1

N_EMBD = int(os.getenv("BDH_N_EMBD", str(default_n_embd)))
N_HEAD = int(os.getenv("BDH_N_HEAD", str(default_n_head)))
N_LAYER = int(os.getenv("BDH_N_LAYER", str(default_n_layer)))
MLP_MULT = int(os.getenv("BDH_MLP_MULT", str(default_mlp_mult)))

if N_EMBD % N_HEAD != 0:
    raise ValueError(f"BDH_N_EMBD ({N_EMBD}) must be divisible by BDH_N_HEAD ({N_HEAD})")

# Build config with the chosen (smaller) model size
BDH_CONFIG = bdh.BDHConfig(
    n_layer=N_LAYER,
    n_embd=N_EMBD,
    n_head=N_HEAD,
    mlp_internal_dim_multiplier=MLP_MULT,
    dropout=0.1,
    vocab_size=256,
 )
config = BDH_CONFIG  # used by the save-model cell

print(
    "Config: "
    f"BLOCK_SIZE={BLOCK_SIZE} BATCH_SIZE={BATCH_SIZE} MAX_ITERS={MAX_ITERS} USE_COMPILE={USE_COMPILE} | "
    f"n_layer={BDH_CONFIG.n_layer} n_embd={BDH_CONFIG.n_embd} n_head={BDH_CONFIG.n_head} mlp_mult={BDH_CONFIG.mlp_internal_dim_multiplier}"
 )

# --------------------
# Prepare training bytes
# --------------------
print("Preparing training data from local Books...")

# Prefer books already loaded earlier, otherwise load from BOOKS_DIR (defined in Step 1)
if "books" not in globals() or not isinstance(books, list) or len(books) == 0:
    if "BOOKS_DIR" not in globals():
        raise NameError("BOOKS_DIR is not defined. Run Step 1 (Configure Local Paths) first.")
    book_paths = sorted(Path(BOOKS_DIR).glob("*.txt"))
    if not book_paths:
        raise FileNotFoundError(f"No .txt files found in {BOOKS_DIR}")
    books = []
    for p in book_paths:
        with open(p, "r", encoding="utf-8", errors="ignore") as f:
            books.append(f.read())
    print(f"✓ Loaded {len(books)} book(s) from {BOOKS_DIR}")

# Write a capped byte-stream to a temp file (memory-safe)
MAX_BYTES = int(os.getenv("BDH_MAX_BYTES", str(1_000_000)))  # ~1MB default cap
temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".bin")
written = 0
try:
    for i, txt in enumerate(books):
        if written >= MAX_BYTES:
            break
        chunk = txt.encode("utf-8", errors="ignore")
        remaining = MAX_BYTES - written
        if remaining <= 0:
            break
        if i > 0 and remaining >= 2:
            temp_file.write(b"\n\n")
            written += 2
            remaining -= 2
        if remaining <= 0:
            break
        if len(chunk) > remaining:
            chunk = chunk[:remaining]
        temp_file.write(chunk)
        written += len(chunk)
finally:
    temp_file.close()

input_file_path = temp_file.name
print(f"✓ Training bytes prepared: {written:,} bytes (cap={MAX_BYTES:,})")

def get_batch(split: str):
    data = np.memmap(input_file_path, dtype=np.uint8, mode="r")
    if len(data) <= BLOCK_SIZE + 1:
        raise ValueError(f"Training data too small ({len(data)} bytes) for BLOCK_SIZE={BLOCK_SIZE}")
    if split == "train":
        data = data[: int(0.9 * len(data))]
    else:
        data = data[int(0.9 * len(data)) :]
    ix = torch.randint(len(data) - BLOCK_SIZE - 1, (BATCH_SIZE,))
    x = torch.stack([torch.from_numpy(data[i : i + BLOCK_SIZE].astype(np.int64)) for i in ix])
    y = torch.stack([torch.from_numpy(data[i + 1 : i + 1 + BLOCK_SIZE].astype(np.int64)) for i in ix])
    if torch.cuda.is_available():
        x = x.pin_memory().to(device, non_blocking=True)
        y = y.pin_memory().to(device, non_blocking=True)
    else:
        x, y = x.to(device), y.to(device)
    return x, y

# --------------------
# Training loop
# --------------------
model = bdh.BDH(BDH_CONFIG).to(device)

# torch.compile can add memory overhead; keep it off by default for small GPUs
if USE_COMPILE:
    try:
        print("Compiling model (optional)...")
        model = torch.compile(model)
        print("✓ Model compiled")
    except Exception as e:
        print(f"⚠️  torch.compile skipped: {e}")
else:
    print("Skipping torch.compile (USE_COMPILE=False)")

optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

x, y = get_batch("train")
loss_acc = 0.0
loss_steps = 0

try:
    for step in range(MAX_ITERS):
        with ctx:
            logits, loss = model(x, y)
        x, y = get_batch("train")
        loss_acc += float(loss.item())
        loss_steps += 1
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad(set_to_none=True)

        if step % LOG_FREQ == 0:
            print(f"Step: {step}/{MAX_ITERS} loss {loss_acc / max(loss_steps, 1):.3f}")
            loss_acc = 0.0
            loss_steps = 0

        if step % 500 == 0 and torch.cuda.is_available():
            torch.cuda.empty_cache()
except RuntimeError as e:
    if "out of memory" in str(e).lower():
        print("\n❌ GPU Out of Memory!")
        print("Try lowering one or more of these env vars and rerun Step 5:")
        print("- BDH_BATCH_SIZE=1")
        print("- BDH_BLOCK_SIZE=64")
        print("- BDH_N_EMBD=96 (and BDH_N_HEAD=2) or BDH_N_EMBD=64")
        print("- BDH_MLP_MULT=8")
        raise
    raise

print("Training done, now generating a sample")
model.eval()
prompt = torch.tensor(bytearray("To be or ", "utf-8"), dtype=torch.long, device=device).unsqueeze(0)
ret = model.generate(prompt, max_new_tokens=100, top_k=3)
ret_decoded = bytes(ret.to(torch.uint8).to("cpu").squeeze(0)).decode(errors="backslashreplace")
print(ret_decoded)

# Cleanup temp file
try:
    os.unlink(input_file_path)
except Exception:
    pass


Using device: cuda (4.1 GB) with dtype bfloat16
Config: BLOCK_SIZE=128 BATCH_SIZE=2 MAX_ITERS=3000 USE_COMPILE=False | n_layer=4 n_embd=128 n_head=2 mlp_mult=16
Preparing training data from local Books...
✓ Training bytes prepared: 1,000,000 bytes (cap=1,000,000)
Skipping torch.compile (USE_COMPILE=False)
Step: 0/3000 loss 5.578
Step: 100/3000 loss 3.067
Step: 200/3000 loss 2.551
Step: 300/3000 loss 2.350
Step: 400/3000 loss 2.201
Step: 500/3000 loss 2.078
Step: 600/3000 loss 2.025
Step: 700/3000 loss 1.972
Step: 800/3000 loss 1.922
Step: 900/3000 loss 1.857
Step: 1000/3000 loss 1.810
Step: 1100/3000 loss 1.819
Step: 1200/3000 loss 1.796
Step: 1300/3000 loss 1.769
Step: 1400/3000 loss 1.711
Step: 1500/3000 loss 1.740
Step: 1600/3000 loss 1.668
Step: 1700/3000 loss 1.668
Step: 1800/3000 loss 1.644
Step: 1900/3000 loss 1.668
Step: 2000/3000 loss 1.617
Step: 2100/3000 loss 1.610
Step: 2200/3000 loss 1.616
Step: 2300/3000 loss 1.580
Step: 2400/3000 loss 1.589
Step: 2500/3000 loss 1.582
Ste

In [18]:
os.makedirs('models', exist_ok=True)

model_path = 'models/bdh_trained.pt'
torch.save({
    'model_state_dict': model.state_dict(),
    'config': config,
    'epochs': 50
}, model_path)

print(f"✅ Model saved to {model_path}")
print(f"   File size: {os.path.getsize(model_path) / 1e6:.1f} MB")


✅ Model saved to models/bdh_trained.pt
   File size: 3.4 MB


## Step 7: Download Model

Download the trained model to your local machine.


In [ ]:
from google.colab import files

print("📥 Downloading trained model...")
files.download(model_path)
print("\n✅ Download complete!")
print("\n📋 Next steps:")
print("1. Move the downloaded file to: Main_System/scripts/models/bdh_trained.pt")
print("2. Run: cd Main_System/scripts && python3 bdh_quickstart.py")
print("3. The script will automatically use the trained model!")
